In [2]:
import os, sys
import pandas as pd

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils import util

In [3]:
def extract_audio_segments(segments_df, segments_dir, sr=32000, segment_sec=5.0, 
                            threshold_factor=0.75, cap_per_class=100):
    """
    Extract high-energy audio segments from audio files with balanced sampling per class.
    
    Args:
        segments_df (pd.DataFrame): DataFrame with 'filename' and 'class_id' columns
        segments_dir (str): Directory containing original audio .wav files
        sr (int): Target sampling rate
        segment_sec (float): Duration of each segment in seconds
        threshold_factor (float): RMS threshold factor for segment filtering
        cap_per_class (int): Maximum segments per class
        
    Returns:
        list: List of segment dictionaries with metadata
    """
    
    # Load and prepare audio files
    audio_files = util.load_audio_files(segments_df, segments_dir, sr, segment_sec, threshold_factor)
    
    if not audio_files:
        print("No valid audio files loaded!")
        return []
    
    print(f"Loaded {len(audio_files)} audio files successfully.")
    
    # Calculate total segments per class for metadata
    class_total_segments = util.calculate_class_totals(audio_files)
    
    # Extract segments with balanced sampling
    segments = util.extract_balanced_segments(audio_files, cap_per_class, segment_sec, sr, class_total_segments)
    
    print(f"Extracted {len(segments)} total segments")
    return segments

In [4]:
# Define Paths
audios_dir = os.path.join('..', 'database', 'audio', 'dev')
dev_data = pd.read_csv(os.path.join('..', 'database', 'meta', 'dev_data.csv'))

# Output directories for extracted segments
segments_output_dir = os.path.join('..', 'database', 'audio_segments')
segments_csv_path = os.path.join('..', 'database', 'meta', 'audio_segments.csv')

# Solo los que estan en dev/ para no encontrar errores
dev_df = dev_data

In [5]:
# Step 1: Extract audio segments and save to disk
util.clean_dir(segments_output_dir)

print("Extracting audio segments...")
segments = extract_audio_segments(
    dev_df, audios_dir, sr=32000, segment_sec=5.0, 
    threshold_factor=0.75, cap_per_class=100
)

print(f"Extracted {len(segments)} segments total")

# Save segments to disk
print("Saving audio segments to disk...")
segments_df = util.save_audio_segments_to_disk(segments, segments_output_dir)
segments_df.to_csv(segments_csv_path, index=False)

print(f"Saved {len(segments_df)} audio segments to {segments_output_dir}")
print(f"Metadata saved to {segments_csv_path}")

Resetting ..\database\audio_segments directory...
Extracting audio segments...
Loaded 474 audio files successfully.
Loaded 474 audio files successfully.
Extracted 2042 total segments
Extracted 2042 segments total
Saving audio segments to disk...
Extracted 2042 total segments
Extracted 2042 segments total
Saving audio segments to disk...
Saved 2042 audio segments to ..\database\audio_segments
Metadata saved to ..\database\meta\audio_segments.csv
Saved 2042 audio segments to ..\database\audio_segments
Metadata saved to ..\database\meta\audio_segments.csv
